In [1]:
!pip install pyyaml h5py

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.15.0


In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])

Epoch 1/10
26/32 [=======================>......] - ETA: 0s - loss: 1.3188 - sparse_categorical_accuracy: 0.6250
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 20ms/step - loss: 1.2468 - sparse_categorical_accuracy: 0.6430 - val_loss: 0.7651 - val_sparse_categorical_accuracy: 0.7680
Epoch 2/10
26/32 [=======================>......] - ETA: 0s - loss: 0.4618 - sparse_categorical_accuracy: 0.8762
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 10ms/step - loss: 0.4434 - sparse_categorical_accuracy: 0.8840 - val_loss: 0.5338 - val_sparse_categorical_accuracy: 0.8370
Epoch 3/10
25/32 [======================>.......] - ETA: 0s - loss: 0.2955 - sparse_categorical_accuracy: 0.9262
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 11ms/step - loss: 0.2878 - sparse_categorical_accuracy: 0.9300 - val_loss: 0.4884 - val_sparse_categorical_accuracy: 0.8360
Epoch 4/10
25/32 [=======

In [6]:
os.listdir(checkpoint_dir)

['cp.ckpt.index', 'checkpoint', 'cp.ckpt.data-00000-of-00001']

In [7]:
model = create_model()
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3087 - sparse_categorical_accuracy: 0.1320 - 460ms/epoch - 14ms/step
Untrained model, accuracy: 13.20%


In [8]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.3995 - sparse_categorical_accuracy: 0.8680 - 167ms/epoch - 5ms/step
Restored model, accuracy: 86.80%


In [9]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

import math
n_batches = len(train_images) / batch_size
n_batches = math.ceil(n_batches)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*n_batches)

model = create_model()

model.save_weights(checkpoint_path.format(epoch=0))

model.fit(train_images,
          train_labels,
          epochs=50,
          batch_size=batch_size,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [10]:
os.listdir(checkpoint_dir)

['cp-0010.ckpt.index',
 'cp-0035.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0050.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0020.ckpt.index',
 'cp-0000.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'checkpoint',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0025.ckpt.index',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001']

In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [12]:
model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.5031 - sparse_categorical_accuracy: 0.8770 - 312ms/epoch - 10ms/step
Restored model, accuracy: 87.70%


In [13]:
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()

model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.5031 - sparse_categorical_accuracy: 0.8770 - 313ms/epoch - 10ms/step
Restored model, accuracy: 87.70%


In [14]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)
model.save('my_model.keras')

Epoch 1/5
32/32 [==============================] - 1s 6ms/step - loss: 1.1216 - sparse_categorical_accuracy: 0.6860
Epoch 2/5
32/32 [==============================] - 0s 6ms/step - loss: 0.4136 - sparse_categorical_accuracy: 0.8820
Epoch 3/5
32/32 [==============================] - 0s 9ms/step - loss: 0.2929 - sparse_categorical_accuracy: 0.9260
Epoch 4/5
32/32 [==============================] - 0s 10ms/step - loss: 0.2015 - sparse_categorical_accuracy: 0.9470
Epoch 5/5
32/32 [==============================] - 0s 9ms/step - loss: 0.1519 - sparse_categorical_accuracy: 0.9650


In [16]:
new_model = tf.keras.models.load_model('my_model.keras')
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 1s - loss: 0.4368 - sparse_categorical_accuracy: 0.8590 - 665ms/epoch - 21ms/step
Restored model, accuracy: 85.90%
32/32 [==============================] - 0s 4ms/step
(1000, 10)


In [18]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 6ms/step - loss: 1.1297 - sparse_categorical_accuracy: 0.6830
Epoch 2/5
32/32 [==============================] - 0s 7ms/step - loss: 0.4036 - sparse_categorical_accuracy: 0.8910
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2822 - sparse_categorical_accuracy: 0.9320
Epoch 4/5
32/32 [==============================] - 0s 6ms/step - loss: 0.1946 - sparse_categorical_accuracy: 0.9530
Epoch 5/5
32/32 [==============================] - 0s 7ms/step - loss: 0.1516 - sparse_categorical_accuracy: 0.9680


In [19]:
!ls saved_model
!ls saved_model/my_model

my_model
assets	fingerprint.pb	keras_metadata.pb  saved_model.pb  variables


In [21]:
new_model = tf.keras.models.load_model('saved_model/my_model')
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4207 - sparse_categorical_accuracy: 0.8650 - 316ms/epoch - 10ms/step
Restored model, accuracy: 86.50%
32/32 [==============================] - 0s 4ms/step
(1000, 10)


In [23]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 7ms/step - loss: 1.1795 - sparse_categorical_accuracy: 0.6480
Epoch 2/5
32/32 [==============================] - 0s 7ms/step - loss: 0.4270 - sparse_categorical_accuracy: 0.8880
Epoch 3/5
32/32 [==============================] - 0s 7ms/step - loss: 0.2787 - sparse_categorical_accuracy: 0.9300
Epoch 4/5
32/32 [==============================] - 0s 7ms/step - loss: 0.2103 - sparse_categorical_accuracy: 0.9510
Epoch 5/5
32/32 [==============================] - 0s 6ms/step - loss: 0.1472 - sparse_categorical_accuracy: 0.9700


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [24]:
new_model = tf.keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               401920    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4411 - sparse_categorical_accuracy: 0.8610 - 459ms/epoch - 14ms/step
Restored model, accuracy: 86.10%
